In [ ]:
import json
import time
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "user", "content": user_prompt},
            ],
        )
        return completion.choices[0].message.content.strip()


def build_prompt(question):
    return f"""
    优化现有问答对中的问题文本，使其更适合知识库检索系统。
    保持原问题的核心含义，同时增强其：
    1. 术语准确性 - 使用保单/手册中的标准术语
    2. 场景完整性 - 补充必要的前提条件
    3. 结构清晰性 - 明确问题类型（如何/是否/需要什么）
    4. 检索友好性 - 包含关键实体和关系
    
    优化原则：
    - 保留原问题的核心意图和实体
    - 使用保单条款/行政手册中的标准表述
    - 补充必要的前提条件（如特定状态、时间限制等）
    - 明确问题类型（操作流程、所需材料、后果影响等）
    
    示例优化：
      原问题：保费扣不了有什么后果？
      优化后：如果保单账户余额不足导致无法扣除月费，会有什么后果？
      
      原问题：改保单信息超时了怎么办？
      优化后：如果保单变更服务未在规定工作日内完成，客户应该采取什么措施？
      
      原问题：小孩能拿死亡赔偿吗？
      优化后：如果未成年受益人没有指定信托人或合法监护人，死亡赔偿款项会如何处理？
      
      原问题：货币转换有什么要求？
      优化后：申请保单货币转换需要满足哪些条件？转换后对附加保障有什么影响？
      
      待优化问题：{question}
      优化后：
    """


def main():
    api_key = "2b064694-cb3b-49e3-bb6e-55ef9ad8351f"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"  
    model_name = "deepseek-r1-distill-qwen-32b-250120"

    input_path = "QA_ds.json"
    output_path = "rewritten.json"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    with open(input_path, "r", encoding="utf-8") as f:
        qa_data = json.load(f)

    rewritten_data = []

    for idx, item in enumerate(qa_data):
        question = item.get("question", "")
        prompt = build_prompt(question)

        try:
            rewritten_question = chat_model.chat(prompt)
        except Exception as e:
            print(f"第 {idx+1} 条问题重写失败：{e}")
            rewritten_question = question

        item["rewrite_question"] = rewritten_question
        rewritten_data.append(item)

        print(f"[{idx+1}/{len(qa_data)}] 原问题: {question} => 重写后: {rewritten_question}")
        time.sleep(1.5)  
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(rewritten_data, f, ensure_ascii=False, indent=2)

    print(f"\n所有问题已重写并保存至 {output_path}")


if __name__ == "__main__":
    main()


[1/222] 原问题: 如果保单持有人去世，如何进行保单持有权的更改？ => 重写后: 优化后的问题：

当保单持有人去世后，法定继承人需要如何进行保单持有权的更改？需要提供哪些文件，如死亡证明和继承权证明，并说明具体的操作流程和所需步骤。
[2/222] 原问题: 如果保单账户余额不足导致无法扣除月费，会有什么后果？ => 重写后: 如果保单账户因余额不足导致月费扣除失败，会有什么后果？
[3/222] 原问题: 如果保单变更服务未在规定工作日内完成，客户应该采取什么措施？ => 重写后: 如果保单变更服务未在规定工作日内完成，客户应该采取什么措施？
[4/222] 原问题: 如何办理宏利投资相连寿险保单的绝对转让手续？需要填写什么表格？ => 重写后: 如何办理宏利投资相连寿险保单的绝对转让手续？申请流程是怎样的？需要填写哪些申请表格？
[5/222] 原问题: 如何取消保单的抵押转让？需要准备哪些材料？ => 重写后: 在保单质押转让已生效且相关贷款已全部偿还的情况下，如何取消该质押转让？取消时需要准备哪些材料？
[6/222] 原问题: 如果投保人具有美国身份或非香港身份，在投保可衍生保单价值的附加保障时需要提供哪些税务合规文件？ => 重写后: 优化后的问题：

如果投保人具有美国税务居民身份或非香港税务居民身份，在投保可衍生保单价值的附加保障时需要提供哪些税务合规文件？

这个优化后的问题：
1. 使用了更准确的术语，如“美国税务居民身份”和“非香港税务居民身份”。
2. 明确了问题类型，即询问所需材料。
3. 包含了关键实体和关系，如投保人、税务合规文件、附加保障、保单价值。
4. 保持了结构清晰，便于检索系统理解和处理。
[7/222] 原问题: 如果已收取的保费高于逾期费用，保单还会失效吗？ => 重写后: 优化后：如果保单已缴纳的保费金额超过应收取的逾期费用，根据保险合同条款，保单是否仍会因未按时缴费而失效？

解释：
1. 术语准确性：使用了“保单”、“保费”、“逾期费用”等标准术语，确保与保单条款一致。
2. 场景完整性：补充了“根据保险合同条款”的前提条件，明确了判断依据。
3. 结构清晰性：使用了“是否”类型的问题，明确询问保单是否会失效。
4. 检索友好性：包含了“保单”、“保费”、“逾期费用”、“保险合同条款”等关键实体，便于知识库